# 分类变量
分类变量是用来表示类别或标记的
与数值变量不同，分类变量是不能排序的(如公司的分类 旅游公司和石油公司无法比较)
因此 分类变量又称为无序变量

## one-hot编码
one-hot就是某一列为1 其他列都为0的方法进行分类 而做出的编码
one-hot编码当分成n类时 需要n列来表示 有k个自由度
但是由于有e1+e2+...+en=1的限制条件 本身只需要k-1个自由度

## 虚拟编码 dummy编码
虚拟编码通过仅使用表示中的k-1个特征来消除额外的自由度

In [ ]:
# one-hot编码 与 dummy编码实践

In [1]:
import pandas as pd
from sklearn import linear_model

In [2]:
# 模拟一个数据集 表示NYC SF Seattle的公寓租金

df = pd.DataFrame({
    'city':['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC','Seattle', 'Seattle', 'Seattle'],
    'rent':[3999, 4000, 4001, 3499, 3500, 3501, 2499,2500,2501]
})

In [3]:
df

,city,rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [4]:
df['rent'].describe()

count       9.000000
mean     3333.333333
std       661.438395
min      2499.000000
25%      2501.000000
50%      3500.000000
75%      3999.000000
max      4001.000000
Name: rent, dtype: float64

In [7]:
# 按照city作为分类变量 创造one-hot矩阵和dummy矩阵

one_hot_df = pd.get_dummies(df,prefix='city')
one_hot_df

,rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [8]:
# 使用drop_first参数可以创建虚拟变量
dummy_df = pd.get_dummies(df, prefix=['city'], drop_first=True)
dummy_df
# 这样的话NYC列被省去 但是仍保证了e1+e2+e3=1

,rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


In [9]:
model_one_hot = linear_model.LinearRegression()
model_one_hot.fit(one_hot_df[['city_NYC','city_SF','city_Seattle']],
                  one_hot_df['rent'])

LinearRegression()

In [10]:
model_one_hot.coef_
# 不同变量的斜率

array([ 166.66666667,  666.66666667, -833.33333333])

In [11]:
model_one_hot.intercept_
# 截距

3333.3333333333335

In [13]:
model_dummy = linear_model.LinearRegression()
model_dummy.fit(dummy_df[['city_SF','city_Seattle']],
                  dummy_df['rent'])

LinearRegression()

In [14]:
model_dummy.coef_

array([  500., -1000.])

In [15]:
model_dummy.intercept_

3500.0

In [ ]:
# 由上述两种编码做线性回归可以看出
# one-hot编码时 截距是所有数据的均值 即当三种分类都不是的时候 就取均值
# dummy编码时 截距是缺省维度的均值 即当不是另外两种的时候 就是剩下那种

## one-hot编码 vs. dummy编码
one-hot和dummy编码十分相似 但是都有各自的优缺点

one-hot编码缺点是有冗余 这会使得同一个问题有多个有效模型

优点是每个特征都有一个明确的分类对应 而不在分类表中的可以使用全0来表示

dummy编码优点是没有冗余 可以生成唯一的可解释模型

缺点是难以处理缺失数据 因为全0已经用来表示参照特征了

在处理大型数据的时候 这两种编码都会过于稀疏而出现问题